__*scraping*__  
page rank <- link들 간의 관계를 이용하여 가중치를 매겨 계산
실제 크롤링을 해서 가중치를 매길 일은 아마 없을 것.... 아마도!
오늘부터 하는 게 더 도움될 것
앞에 했던 건 빅데이터 수집의 관점, 어떻게 바라봐야 하는지
__*data scraping / data crawling*__
scraping은 안 한 건 아니고 하긴 했음 검색결과 해서 우리가 원하는 거 뜯어오는 거
둘의 차이를 보면 후자는 스케일이 굉장히 크다. 웹사이트를 돌고 돌아서 왔다갔다 할 거니까. 큰 규모
당연히 우리가 빅데이터라고 부르면 후자에 해당 (volumn이라고 하는 관점에서)
구글 첫 페이지 긁어오거나 네이버, 다음 검색 첫페이지 긁어오기...딱 내가 원하는 것만큼만 <- 전자
웹페이지를 통으로 긁어와서 저장하는 부분이 반드시 있어야 함 <- 후자
전자는 그 부분이 필요없음. 내가 원하는 부분만 들고와서 쓴 다음에 나머지는 버려버리면 됨
후자는 crawler agent가 필요 (어제 만들었던 걔)
전자는 parser가 필요 <- 내가 원하는 데이터가 어디 있는지 찾은 다음에 걔만 가져와야 되니까
둘다 파서가 필요하긴 한데 굳이 구분하자면?
우리가 크롤러를 만들어서 seed 주소를 만들면 해당 링크 방문 list를 build하고 중간에 부산물로
전체 페이지를 다운로드 다운로드 해서 자연어 처리나 이런 걸 거쳐서 indexing이라고 하는 데이터베이스를 만들 것
web scraping
특정 사이트 하나 들어가서 우리가 만든 scraper가 scraping할 건데 어떤 걸? xml, sql, excel
web crawler가 indexing하는 거랑 어떻게 보면 똑같음
크롤러의 일부분이라고 볼 수도 있음
크롤링, 스크래핑이랑 조금씩 다름 - 목적이 다름
크롤링은 앵커라고 하는 텍스트를 찾아서 네트워크를 끊임없이 돌아다님
스크래핑은 특정 페이지의 내용을 내 목적에 맞게 가져옴

Architecture
멀티 스레드로 돌면서 인터넷으로부터 웹페이지를 다운받음 <- URL주소를 가지고
큐라는 구조로 어제는 스케쥴러를 대신함
스케쥴러에 의해서 시드 주소를 주면 웹페이지를 하나 긁어오고 거기서 우리가 원하는 데이터를 따로 저장해놓고
다시 다음페이지로 감
어제 그림보다 훨씬 간단
어제 했던 DB를 안 버린 상태에서 오늘 table을 새로 만들 것

In [1]:
import requests
import time

headers = {"user-agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

# request package를 사용하여 error handling
def download(method, url, param=None, data=None, timeout=1, maxretries=3):
    try:
        resp = requests.request(method, url, params=param, data=data, headers=headers)
        # 한글처럼 아스키가 아닌 문자들을 변환시킬 때 서비스키도 같이 바꿔버림
        resp.raise_for_status()
        #if(resp.headers['Content-Type'].split("/")[0] == "image"):
            #resp.headers['Content-Type'].split("/")[1] #확장자
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0:
            print(e.response.status_code)
            print(e.response.reason)
            time.sleep(timeout) # 1초 정도 기다림
            print(maxretries)
            print("재시도") #maxretries
            download(method, url, param, data, timeout, maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [2]:
from bs4 import BeautifulSoup
import re
import sqlite3

In [39]:
url = "http://www.ppomppu.co.kr/zboard/zboard.php"
param = {
    "id":"ppomppu",
    "page":1
}
resp = download("get", url, param = param)
dom = BeautifulSoup(resp.text, 'lxml')

In [123]:
requests.compat.urlparse(resp.request.url)

ParseResult(scheme='http', netloc='www.ppomppu.co.kr', path='/zboard/zboard.php', params='', query='id=freeboard', fragment='')

In [131]:
requests.compat.urlparse(resp.request.url)[4]

'id=freeboard'

In [71]:
for tr in dom.select("tr[class^=list]")[1:]:
    # tr인데 class가 list로 시작하는
    td = tr.find_all("td", recursive=False)
    print(requests.compat.urljoin(url, td[3].a.img["src"]))
    print(td[3].select("td[valign=middle] > a")[0].text)
    temp = td[5].text.split("-")
    print((0, 0) if len(temp) < 2 else ",".join(temp))
    print(td[6].text.strip())
    print()

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]

[]



In [66]:
[requests.compat.urljoin(resp.request.url, _["href"]) for _ in dom.select("td#page_list a")]

['http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=2&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=3&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=4&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=5&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=6&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=7&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=8&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=9&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=10&divpage=1221',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=11&divpage=1221']

In [156]:
# 아래아래 코드(선생님)와 뜻은 같음
def addlink(seed):
    ppomppulink = []
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    for tr in dom.select("tr[class^=list]")[1:]:
        a = tr.find_all("a")
        if len(a) > 1:
            ppomppulink.append(a[1]["href"])
    return(ppomppulink)

In [154]:
[requests.compat.urljoin(resp.request.url, _) for _ in ppomppulink]

['http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543168',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543167',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543166',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543165',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543164',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543163',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543162',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543161',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543160',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543159',
 'http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard&page=1&divpage=1221&no=6543158',

In [167]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    return [requests.compat.urljoin(seed, _.find_parent()["href"]) for _ in dom.select("font.list_title") if _.find_parent() and _.find_parent().has_attr("href")]

In [169]:
def parseContent(url):
    html = download("get", url)
    dom = BeautifulSoup(html.text, "lxml")
    
    return {"title":dom.select_one("font.view_title2").text.strip(),
           "body":dom.select_one("td.board-contents").text.strip(),
           "comments":[_.text.strip() for _ in dom.select("div[id^=commentContent_]")]}

In [150]:
url = "http://www.ppomppu.co.kr/zboard/zboard.php"
param = {
    "id":"freeboard",
}
resp = download("get", url, param = param)
dom = BeautifulSoup(resp.text, 'lxml')

In [180]:
seed = list()
seed.extend([requests.compat.urljoin(resp.request.url, _["href"]) 
             for _ in dom.select("td#page_list a")])
seen=list()
queue=list()
linkList = list()
# print(seed)

while seed:
    baseURL = seed.pop(0)
    seen.append(baseURL)
    linkList = parseURL(baseURL)
    
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))
    
#     html = download("get", baseURL)
#     dom = BeautifulSoup(html.text, "lxml")
#     for tr in dom.select("tr[class^=list]")[1:]:
#         a = tr.find_all("a")
#         if len(a) > 1:
#             linkList.append(a[1]["href"])
    
#     print(len(seed), len(seen), len(linkList))

Queue: 30, Seen: 1
Queue: 60, Seen: 2
Queue: 90, Seen: 3
Queue: 120, Seen: 4
Queue: 150, Seen: 5
Queue: 180, Seen: 6
Queue: 210, Seen: 7
Queue: 240, Seen: 8
Queue: 270, Seen: 9
Queue: 300, Seen: 10
Queue: 330, Seen: 11


In [174]:
con = sqlite3.connect("bot.db")
cur = con.cursor()

In [179]:
cur.executescript("""
    DROP TABLE IF EXISTS table3;
    CREATE TABLE table3(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title TEXT NOT NULL,
        content TEXT NOT NULL,
        data TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
    
    DROP TABLE IF EXISTS table4;
    CREATE TABLE table4(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table3_id INTEGER NOT NULL,
        comment TEXT NOT NULL
    );
""")
con.commit()

In [181]:
# contents=list()
while(queue):
    baseURL = queue.pop()
    seen.append(baseURL)
    content = parseContent(baseURL)
    cur.execute("""
        INSERT INTO table3 (title, content)
        VALUES(?,?)
    """, [content["title"], content["body"]])
    con.commit()
    
    # 원래는 이렇게 비교하는 거 안좋음. 키를 가져와서 비교하는 게 좋음
    cur.execute("""
        SELECT id
        FROM table3
        WHERE title=? AND content=?;
    """, [content["title"], content["body"]])
    con.commit()
    table3_id = cur.fetchone()[0]
    
    for comment in content["comments"]:
        cur.execute("""
            INSERT INTO table4 (table3_id, comment)
            VALUES(?,?)
        """, [table3_id, comment])
    con.commit()
#     contents.append(parseContent(currentURL))
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))

Queue: 329, Seen: 12
Queue: 328, Seen: 13
Queue: 327, Seen: 14
Queue: 326, Seen: 15
Queue: 325, Seen: 16
Queue: 324, Seen: 17
Queue: 323, Seen: 18
Queue: 322, Seen: 19
Queue: 321, Seen: 20
Queue: 320, Seen: 21
Queue: 319, Seen: 22
Queue: 318, Seen: 23
Queue: 317, Seen: 24
Queue: 316, Seen: 25
Queue: 315, Seen: 26
Queue: 314, Seen: 27
Queue: 313, Seen: 28
Queue: 312, Seen: 29
Queue: 311, Seen: 30
Queue: 310, Seen: 31
Queue: 309, Seen: 32
Queue: 308, Seen: 33
Queue: 307, Seen: 34
Queue: 306, Seen: 35
Queue: 305, Seen: 36
Queue: 304, Seen: 37
Queue: 303, Seen: 38
Queue: 302, Seen: 39
Queue: 301, Seen: 40
Queue: 300, Seen: 41
Queue: 299, Seen: 42
Queue: 298, Seen: 43
Queue: 297, Seen: 44
Queue: 296, Seen: 45
Queue: 295, Seen: 46
Queue: 294, Seen: 47
Queue: 293, Seen: 48
Queue: 292, Seen: 49
Queue: 291, Seen: 50
Queue: 290, Seen: 51
Queue: 289, Seen: 52
Queue: 288, Seen: 53
Queue: 287, Seen: 54
Queue: 286, Seen: 55
Queue: 285, Seen: 56
Queue: 284, Seen: 57
Queue: 283, Seen: 58
Queue: 282, S

In [182]:
con.close()
cur.close()

ProgrammingError: Cannot operate on a closed database.

In [173]:
contents[0]

{'title': '불매운동 0순위',
 'body': '일본차도 안타고, 일본여행도 안갈거고, 유니클로도 안살거고, 카메라도 안살거고\n\xa0\n\n제일중요한게 빠졌네요\n\n--------------------------------------------------------------------------------------------------------------------\n\n내컴퓨터 > C:\\Windows\\System32\\drivers\\etc 들어가시면 hosts 파일이있습니다.\n\nhosts파일 열기(메모장등 편집툴로)\n\n\n127.0.0.1 www.chosun.com\n\n\n127.0.0.1 biz.chosun.com\n\n\n127.0.0.1 blogs.chosun.com\n\n\n127.0.0.1 cnnews.chosun.com\n\n\n127.0.0.1 danmee.chosun.com\n\n\n127.0.0.1 economy.chosun.com\n\n\n127.0.0.1 edu.chosun.com\n\n\n127.0.0.1 english.chosun.com\n\n\n127.0.0.1 fashion.chosun.com\n\n\n127.0.0.1 health.chosun.com\n\n\n127.0.0.1 issue.chosun.com\n\n\n127.0.0.1 japanese.chosun.com\n\n\n127.0.0.1 kartrider.chosun.com\n\n\n127.0.0.1 kid.chosun.com\n\n\n127.0.0.1 lifenlearning.chosun.com\n\n\n127.0.0.1 m.chosun.com\n\n\n127.0.0.1 m.health.chosun.com\n\n\n127.0.0.1 marathon.chosun.com\n\n\n127.0.0.1 mindgil.chosun.com\n\n\n127.0.0.1 monthly.chosun.com\n\n\n127.0.0.1 news.chosun.co

In [138]:
requests.compat.urlparse(resp.request.url)

ParseResult(scheme='http', netloc='www.ppomppu.co.kr', path='/zboard/zboard.php', params='', query='id=freeboard&page=11', fragment='')